In [12]:
import pandas as pd
import json
import sqlite3

df_movies = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")
df_credits = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")

pd.set_option('display.max_columns', None)

In [13]:
for col in df_movies.columns:
    print(col)

budget
genres
homepage
id
keywords
original_language
original_title
overview
popularity
production_companies
production_countries
release_date
revenue
runtime
spoken_languages
status
tagline
title
vote_average
vote_count


In [14]:
for col in df_credits.columns:
    print(col)

movie_id
title
cast
crew


In [15]:
df_credits.drop(['title'], axis=1, inplace=True)

In [16]:
df_credits.shape

(4803, 3)

In [17]:
df_movies.shape

(4803, 20)

In [18]:
conn = sqlite3.connect(':memory:')

# Guardar los DataFrames como tablas en la base de datos SQLite
df_movies.to_sql('movies', conn, index=False)
df_credits.to_sql('credits', conn, index=False)

# Ejecutar una consulta SQL para combinar los datos de ambas tablas
query = '''
        SELECT *
        FROM movies
        INNER JOIN credits ON movies.id = credits.movie_id
        '''

# Ejecutar la consulta y obtener el resultado como un DataFrame
df_combined = pd.read_sql(query, conn)

In [19]:
df_combined.shape

(4803, 23)

In [20]:
df_combined.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [21]:
df_combined.head(2)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [22]:
df_combined.drop(['budget', 'homepage', 'original_language', 'tagline', 'original_title', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'vote_average', 'vote_count', 'id'], axis=1, inplace=True)

In [23]:
def extraer_nombres(lista_diccionarios, limit=None):
    try:
        # Convertir la cadena JSON a una lista de diccionarios
        lista_diccionarios = json.loads(lista_diccionarios)
        # Si no se especifica un límite, extraer todos los nombres
        if limit is None:
            nombres = [item['name'] for item in lista_diccionarios]
        else:
            # Si se especifica un límite, extraer solo los primeros 'limit' nombres
            nombres = [item['name'] for item in lista_diccionarios[:limit]]
        return nombres
    except (json.JSONDecodeError, KeyError):
        # En caso de error o si la clave 'name' no está presente, retornar una lista vacía
        return []
    
# Aplicar la función a todas las columnas, excepto a 'cast'
columnas_json = ['genres', 'keywords']
for columna in columnas_json:
    df_combined[columna] = df_combined[columna].apply(extraer_nombres)

# Aplicar la función a la columna 'cast', limitando a los primeros tres nombres
df_combined['cast'] = df_combined['cast'].apply(extraer_nombres, limit=3)

In [24]:
def extract_director_name(crew_json):
    try:
        # Convertir la cadena JSON a una lista de diccionarios
        crew_list = json.loads(crew_json)
        
        # Buscar el diccionario con 'job' igual a 'Director'
        director_dict = next((item for item in crew_list if item['job'] == 'Director'), None)
        
        # Si se encuentra el diccionario del director, devolver su nombre, de lo contrario devolver None
        if director_dict:
            return director_dict['name']
        else:
            return None
    except (json.JSONDecodeError, KeyError):
        # En caso de error o si la columna no está en formato JSON, retornar None
        return None
    
df_combined['crew'] = df_combined['crew'].apply(lambda x: [extract_director_name(x)])

In [25]:
df_combined.shape

(4803, 7)

In [26]:
# Convertir el contenido de la columna 'overview' en listas de palabras
df_combined['overview'] = df_combined['overview'].astype(str)
df_combined['overview'] = df_combined['overview'].apply(lambda x: x.split())

In [27]:
pd.set_option('display.max_colwidth', None)

In [28]:
def reemplazar_nulos_con_listas_vacias(df, columnas):
    for columna in columnas:
        df_combined['genres'] = df_combined['genres'].apply(lambda x: [] if pd.isnull(x) else x)
        df_combined['cast'] = df_combined['cast'].apply(lambda x: [] if pd.isnull(x) else x)
        df_combined['crew'] = df_combined['crew'].apply(lambda x: [] if pd.isnull(x) else x)
        df_combined['overview'] = df_combined['overview'].apply(lambda x: [] if pd.isnull(x) else x)
    return df_combined

In [29]:
def eliminar_espacios(texto):
    if texto is not None:
        # Si el texto es una cadena de caracteres, eliminar los espacios entre palabras
        if isinstance(texto, str):
            return ''.join(texto.split())
        # Si el texto es una lista, eliminar los espacios entre palabras en cada elemento
        elif isinstance(texto, list):
            return [''.join(palabra.split()) if isinstance(palabra, str) else palabra for palabra in texto]
    # Si el texto es None, devolver None sin cambios
    return texto


df_combined['genres'] = df_combined['genres'].apply(eliminar_espacios)
df_combined['cast'] = df_combined['cast'].apply(eliminar_espacios)
df_combined['crew'] = df_combined['crew'].apply(eliminar_espacios)
df_combined['keywords'] = df_combined['keywords'].apply(eliminar_espacios)

In [30]:
df_combined.head(2)

,genres,keywords,overview,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, society, spacetravel, futuristic, romance, space, alien, tribe, alienplanet, cgi, marine, soldier, battle, loveaffair, antiwar, powerrelations, mindandsoul, 3d]","[In, the, 22nd, century,, a, paraplegic, Marine, is, dispatched, to, the, moon, Pandora, on, a, unique, mission,, but, becomes, torn, between, following, orders, and, protecting, an, alien, civilization.]",Avatar,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatradingcompany, loveofone'slife, traitor, shipwreck, strongwoman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercreditsstinger]","[Captain, Barbossa,, long, believed, to, be, dead,, has, come, back, to, life, and, is, headed, to, the, edge, of, the, Earth, with, Will, Turner, and, Elizabeth, Swann., But, nothing, is, quite, as, it, seems.]",Pirates of the Caribbean: At World's End,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [31]:
df_combined.shape

(4803, 7)

In [32]:
df_combined['tags'] = df_combined.apply(lambda row: ' '.join(row.astype(str)), axis=1)
df_combined['tags'] = df_combined['tags'].str.replace(r"[\[\]\']", '', regex=True)
df_combined['tags'] = df_combined['tags'].str.replace(',', ' ')

In [33]:
import re

# Función para limpiar los signos de puntuación
def limpiar_signos(texto):
    # Utilizar una expresión regular para eliminar los signos de puntuación
    texto_limpio = re.sub(r'[^\w\s]', '', texto)
    return texto_limpio

df_combined['tags'] = df_combined['tags'].apply(limpiar_signos)
df_combined.head(3)

,genres,keywords,overview,title,movie_id,cast,crew,tags
0,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, society, spacetravel, futuristic, romance, space, alien, tribe, alienplanet, cgi, marine, soldier, battle, loveaffair, antiwar, powerrelations, mindandsoul, 3d]","[In, the, 22nd, century,, a, paraplegic, Marine, is, dispatched, to, the, moon, Pandora, on, a, unique, mission,, but, becomes, torn, between, following, orders, and, protecting, an, alien, civilization.]",Avatar,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d In the 22nd century a paraplegic Marine is dispatched to the moon Pandora on a unique mission but becomes torn between following orders and protecting an alien civilization Avatar 19995 SamWorthington ZoeSaldana SigourneyWeaver JamesCameron
1,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatradingcompany, loveofone'slife, traitor, shipwreck, strongwoman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercreditsstinger]","[Captain, Barbossa,, long, believed, to, be, dead,, has, come, back, to, life, and, is, headed, to, the, edge, of, the, Earth, with, Will, Turner, and, Elizabeth, Swann., But, nothing, is, quite, as, it, seems.]",Pirates of the Caribbean: At World's End,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],Adventure Fantasy Action ocean drugabuse exoticisland eastindiatradingcompany loveofoneslife traitor shipwreck strongwoman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger Captain Barbossa long believed to be dead has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann But nothing is quite as it seems Pirates of the Caribbean At Worlds End 285 JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski
2,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, britishsecretservice, unitedkingdom]","[A, cryptic, message, from, Bond’s, past, sends, him, on, a, trail, to, uncover, a, sinister, organization., While, M, battles, political, forces, to, keep, the, secret, service, alive,, Bond, peels, back, the, layers, of, deceit, to, reveal, the, terrible, truth, behind, SPECTRE.]",Spectre,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],Action Adventure Crime spy basedonnovel secretagent sequel mi6 britishsecretservice unitedkingdom A cryptic message from Bonds past sends him on a trail to uncover a sinister organization While M battles political forces to keep the secret service alive Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE Spectre 206647 DanielCraig ChristophWaltz LéaSeydoux SamMendes


In [34]:
df_combined.shape

(4803, 8)

In [35]:
df_combined['tags'][0]

'Action  Adventure  Fantasy  ScienceFiction cultureclash  future  spacewar  spacecolony  society  spacetravel  futuristic  romance  space  alien  tribe  alienplanet  cgi  marine  soldier  battle  loveaffair  antiwar  powerrelations  mindandsoul  3d In  the  22nd  century   a  paraplegic  Marine  is  dispatched  to  the  moon  Pandora  on  a  unique  mission   but  becomes  torn  between  following  orders  and  protecting  an  alien  civilization Avatar 19995 SamWorthington  ZoeSaldana  SigourneyWeaver JamesCameron'

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorización de texto
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df_combined['tags'])

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

# Cálculo de similitud coseno
similarity = cosine_similarity(vectors)

In [38]:
import numpy as np

# Encuentra los índices de los vecinos más cercanos para cada instancia
k = 5  # Número de vecinos más cercanos
nearest_neighbors = np.argpartition(similarity, -k, axis=1)[:, -k:]

In [39]:
# Ejemplo de predicción para la primera instancia de prueba
indice_instancia_prueba = 0
vecinos_mas_cercanos = nearest_neighbors[indice_instancia_prueba]

# Obtén las etiquetas asociadas con los vecinos más cercanos
etiquetas_vecinos = df_combined['tags'].iloc[vecinos_mas_cercanos].tolist()

# Calcula la etiqueta más común entre los vecinos más cercanos
from collections import Counter
etiqueta_predicha = Counter(etiquetas_vecinos).most_common(1)[0][0]

In [40]:
def recommend(movie):
    # Encuentra el índice de la película en tu DataFrame
    movie_index = df_combined[df_combined["title"] == movie].index[0]
    
    # Calcula las distancias de similitud entre la película de entrada y todas las demás
    distances = similarity[movie_index]
    
    # Encuentra los índices de las películas más similares (excepto la película de entrada)
    movie_indices = sorted(range(len(distances)), key=lambda i: distances[i], reverse=True)[1:6]
    
    # Imprime los títulos de las películas más similares
    for index in movie_indices:
        print(df_combined.iloc[index]["title"])

In [43]:
recommend('Avatar')

Aliens
The Marine
Apollo 18
Star Trek Into Darkness
Titan A.E.
